In [1]:
import ee

ee.Authenticate()
ee.Initialize(project='cibravo2')

In [5]:
import geopandas as gpd

data = gpd.read_file('data_center.geojson').to_crs(4326)
data.head()

,Name,geometry
0,BARQUILLO (MEF03-101296),POINT Z (-72.97009 -41.56678 0.00000)
1,BOLADOS (MEF04-110454),POINT Z (-74.18728 -44.34708 0.00000)
2,CONCOTO (EA48 - 110466),POINT Z (-73.80610 -44.16267 0.00000)
3,CANAL PEREZ NORTE (EA41 - 110523),POINT Z (-73.84893 -44.14898 0.00000)
4,ELENA (EA40 - 110510),POINT Z (-73.81244 -44.25456 0.00000)


In [18]:
data[data['Name'] == 'RODRIGUEZ (EA15-110823)']

,Name,geometry
29,RODRIGUEZ (EA15-110823),POINT Z (-73.50121 -45.37232 0.00000)


In [7]:
import json

llaguepe = data[data['Name'] == 'RODRIGUEZ (EA15-110823)']
aoi = gpd.GeoSeries(llaguepe['geometry'].buffer(0.03)).to_crs(4326).to_json()
aoi_bbox = ee.Geometry.Polygon(json.loads(aoi)['features'][0]['geometry']['coordinates'], 'EPSG:4326')


C:\Users\jacmo\AppData\Local\Temp\ipykernel_15144\1354604125.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi = gpd.GeoSeries(llaguepe['geometry'].buffer(0.03)).to_crs(4326).to_json()


In [8]:
import shapely 

def apply_scale_factors(image):

    image = image.clip(aoi_bbox)
    optical_bands = image.select('B.').multiply(2.75e-05).add(-0.2)
    return image.addBands(optical_bands, None, True)

def add_NDVI_NDBI(image):
    ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
    ndbi = image.normalizedDifference(['SR_B7', 'SR_B4']).rename('NDBI')
    return image.addBands(ndvi, None, True)\
                .addBands(ndbi, None, True)

In [9]:
import json 

aoi = gpd.GeoSeries(llaguepe['geometry'].buffer(0.1)).to_crs(4326).to_json()
aoi_bbox = ee.Geometry.Polygon(json.loads(aoi)['features'][0]['geometry']['coordinates'], 'EPSG:4326')

C:\Users\jacmo\AppData\Local\Temp\ipykernel_15144\3732804932.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi = gpd.GeoSeries(llaguepe['geometry'].buffer(0.1)).to_crs(4326).to_json()


In [101]:
def getImage(aoi, dates, clouds=10, mosaic=False):
    collection = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(aoi).filterDate(dates[0], dates[1])
    collection = collection.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', clouds))
    collection = collection.map(preprocess)
    collection = collection.sort('system:time_start', False)
    images = collection.aggregate_array('system:id').getInfo()
    print(f'Se obtuvieron {len(images)} imágenes')
    return collection, images

def preprocess(image):

    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    optical_bands = image.select('B.').multiply(0.0001)
    return image.addBands(ndvi).addBands(optical_bands, None, True)


dates = ['2023-12-11', '2024-12-12']
collection, images = getImage(aoi_bbox, dates, clouds=1, mosaic=True)

Se obtuvieron 4 imágenes


In [102]:
# image = ee.Image(images[0])
image = collection.median()
image = image.clip(aoi_bbox)

In [104]:
import geemap

# -73.50121 -45.37232
Map = geemap.Map(center=[-45.37232, -73.50121], zoom=16)
Map.addLayer(aoi_bbox)


# ndwi = image.select(['B3'])
# ndwiViz = {'palette': ['00FFFF', '0000FF']}
# Map.addLayer(ndwi, ndwiViz, 'NDWI', True)


# imageRGB = image.visualize(**{'bands': ['B2', 'B3', 'B4'],'min':0, 'max': 0.5})
imageRGB = image.visualize(**{'bands': ['TCI_R', 'TCI_G', 'TCI_B'],'min':0, 'max': 100})
Map.addLayer(imageRGB, {}, 'imageRGB', True)


Map

Map(center=[-45.37232, -73.50121], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

In [73]:
import matplotlib.pyplot as plt
import contextily as ctx
import numpy as np
import geemap
import pandas as pd
import geopandas as gpd
import json
from scipy.interpolate import griddata
from scipy.spatial import Voronoi, voronoi_plot_2d
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from shapely.geometry import Point, Polygon
import rasterio
from rasterio.plot import show, show_hist
# from osgeo import gdal
import seaborn as sns
import rioxarray
from sklearn.linear_model import LinearRegression
from pysal.model import spreg
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
import pysal as ps
from libpysal import weights
import esda
import warnings
from libpysal.weights.raster import da2WSP
from splot.esda import moran_scatterplot, plot_local_autocorrelation

warnings.filterwarnings('ignore')

In [78]:
fecha_inicio_0 = '2018-01-01'
fecha_final_0 = '2022-02-28'
aoi = aoi_bbox

In [79]:
# Función que retorna la colección de imágenes de MODIS
def get_modis_collection(startDate, endDate):
    modis_collection = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI')\
    .filterDate(startDate, endDate)\
    .select("chlor_a")\

    return modis_collection

# Función que retorna las fechas de toda la colección
def get_modis_dates(modis_collection):
    dates = modis_collection.aggregate_array('system:time_start').getInfo()
    dates = [pd.to_datetime(date, unit='ms') for date in dates]
    dates = [date.strftime('%Y-%m-%d') for date in dates]

    return dates

def get_sentinel2_collection(startDate, endDate, aoi):
    sentinel2_collection = ee.ImageCollection('COPERNICUS/S2_SR')\
    .filterDate(startDate, endDate)\
    .filterBounds(aoi)

    return sentinel2_collection

In [80]:
# Obtenemos la colección de imágenes de MODIS y verificamos las fechas incluidas.

test_modis_collection = get_modis_collection(fecha_inicio_0, fecha_final_0)
test_modis_dates = get_modis_dates(test_modis_collection)

# test_modis_dates

In [81]:
# Obtenemos la colección de imágenes de Sentinel 2 y verificamos las fechas incluidas. 
# Filtramos para obtener sólo imagenes de excelente calidad en cuanto a presencia de nubes.

sentinel2_collection = get_sentinel2_collection(fecha_inicio_0, fecha_final_0, aoi).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 0.2)
test_sentinel_dates = get_modis_dates(sentinel2_collection)

# test_sentinel_dates

In [82]:
'''
Se calculan las fechas para las que existen imagenes
disponibles en ambos satélites. 
'''

fechas = set(test_modis_dates) & set(test_sentinel_dates)

# fechas

In [83]:
'''
NO RECOMENDAMOS EJECUTAR ESTA CELDA.
Se obtiene cada una de las imágenes para el satélite MODIS-AQUA.
Se almacenan localmente en disco para luego poder ser poligonizadas a través de GDAL.
'''

for fecha in fechas:
    dayOfInterest = ee.Date(fecha)
    # Obtenemos la imagen del satélite MODIS para ese día

    modisCollection = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI')\
    .filterDate(dayOfInterest, dayOfInterest.advance(1, 'day'))\
    .filterBounds(aoi)\
    .select("chlor_a").first()

    geemap.ee_export_image(modisCollection, scale=4616, filename=f'MODIS/{fecha}.tif', region=aoi)  

    
    

Generating URL ...
Please wait ...
Data downloaded to c:\Users\jacmo\OneDrive - uc.cl\Documentos\cosas de la u\17. XVII\4. CDG\Proyecto\MODIS\2019-08-19.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\jacmo\OneDrive - uc.cl\Documentos\cosas de la u\17. XVII\4. CDG\Proyecto\MODIS\2019-02-03.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\jacmo\OneDrive - uc.cl\Documentos\cosas de la u\17. XVII\4. CDG\Proyecto\MODIS\2020-04-08.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\jacmo\OneDrive - uc.cl\Documentos\cosas de la u\17. XVII\4. CDG\Proyecto\MODIS\2020-11-19.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\jacmo\OneDrive - uc.cl\Documentos\cosas de la u\17. XVII\4. CDG\Proyecto\MODIS\2022-02-22.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\jacmo\OneDrive - uc.cl\Documentos\cosas de la u\17. XVII\4. CDG\Proyecto\MODIS\2021-12-09.tif
Generating URL ...
Please wait ...
Data downloaded to c:\U

In [98]:
aoi.toGeoJSONString()
# gpd.GeoDataFrame([aoi.toGeoJSON])

EEException: Cannot convert a computed geometry to GeoJSON. Wrap a getInfo() call in json.dumps instead.

In [84]:
# Enhanced vegetation index (EVI)

def calculate_evi(collection_images):
 
    evi_collection = collection_images.map(lambda image: image.expression(
        '(2.5 * (NIR - RED)) / (NIR + 6 * RED - 7.5 * BLUE + 1)',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }
    ))

    return evi_collection

# Normalized Ratio Vegetation Index (NRVI)
def calculate_nrvi(collection_images):
    nrvi_collection = collection_images.map(lambda image: image.expression(
        '(RED/NIR - 1) / (RED/NIR + 1)',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4')
        }
    ))

    return nrvi_collection

# Normalized Difference Water Index (NDWI)
def calculate_ndwi(collection_images):
    ndwi_collection = collection_images.map(lambda image: image.expression(
        '(GREEN - NIR) / (GREEN + NIR)',
        {
            'NIR': image.select('B8'),
            'GREEN': image.select('B3')
        }
    ))

    return ndwi_collection

# Normalized Difference Water Index (NDWI2)   ndvi_2 es un typo
def calculate_ndvi2(collection_images):
    ndvi2_collection = collection_images.map(lambda image: image.expression(
        '(NIR - SWIR2) / (NIR + SWIR2)',
        {
            'NIR': image.select('B8'),
            'SWIR2': image.select('B12')
        }
    ))

    return ndvi2_collection

# Modified Normalized Difference Water Index (MNDWI)
def calculate_mndwi(collection_images):
    mndwi_collection = collection_images.map(lambda image: image.expression(
        '(GREEN - SWIR2) / (GREEN + SWIR2)',
        {
            'GREEN': image.select('B3'),
            'SWIR2': image.select('B12')
        }
    ))

    return mndwi_collection

# Green Normalized Difference Vegetation Index
def calculate_gndvi(collection_images):
    gndvi_collection = collection_images.map(lambda image: image.expression(
        '(NIR - GREEN) / (NIR + GREEN)',
        {
            'NIR': image.select('B8'),
            'GREEN': image.select('B3')
        }
    ))

    return gndvi_collection

# Normalized Difference Vegetation Index
def calculate_ndvi(collection_images):
    ndvi_collection = collection_images.map(lambda image: image.expression(
        '(NIR - RED) / (NIR + RED)',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4')
        }
    ))

    return ndvi_collection

# Indices de bandas

def calculate_b3b1(collection_images):
    b3b1_collection = collection_images.map(lambda image: image.expression(
        'GREEN / AEROSOLS',
        {
            'GREEN': image.select('B3'),
            'AEROSOLS': image.select('B1')
        }
    ))

    return b3b1_collection


def calculate_b3b2(collection_images):
    b3b2_collection = collection_images.map(lambda image: image.expression(
        'GREEN / BLUE',
        {
            'GREEN': image.select('B3'),
            'BLUE': image.select('B2')
        }
    ))

    return b3b2_collection

def calculate_b3b4(collection_images):
    b3b4_collection = collection_images.map(lambda image: image.expression(
        'GREEN / RED',
        {
            'GREEN': image.select('B3'),
            'RED': image.select('B4')
        }
    ))

    return b3b4_collection

def calculate_b5b4(collection_images):
    b5b4_collection = collection_images.map(lambda image: image.expression(
        'RED_EDGE_1 / RED',
        {
            'RED_EDGE_1': image.select('B5'),
            'RED': image.select('B4')
        }
    ))

    return b5b4_collection

def calculate_b6b4(collection_images):
    b6b4_collection = collection_images.map(lambda image: image.expression(
        'RED_EDGE_2 / RED',
        {
            'RED_EDGE_2': image.select('B6'),
            'RED': image.select('B4')
        }
    ))

    return b6b4_collection

def calculate_b7b4(collection_images):
    b7b4_collection = collection_images.map(lambda image: image.expression(
        'RED_EDGE_3 / RED',
        {
            'RED_EDGE_3': image.select('B7'),
            'RED': image.select('B4')
        }
    ))

    return b7b4_collection

def calculate_b8ab4(collection_images):
    b8ab4_collection = collection_images.map(lambda image: image.expression(
        'NIR / RED',
        {
            'NIR': image.select('B8A'),
            'RED': image.select('B4')
        }
    ))

    return b8ab4_collection

def calculate_b8b4(collection_images):
    b8b4_collection = collection_images.map(lambda image: image.expression(
        'NIR / RED',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4')
        }
    ))

    return b8b4_collection

def calculate_oc2(collection_images):
    oc2_collection = collection_images.map(lambda image: image.expression(
        '10 ** (0.319 - 2.336 * R_2 + 0.879 * (R_2**2) - 0.135 * (R_2**3)) - 0.071',
        {
            'R_2': ee.Image(image.select('B2')).divide(image.select('B3')).log10()
        }
    ))
    
    return oc2_collection

def calculate_oc4(collection_images):
    oc4_collection = collection_images.map(lambda image: image.expression(
        '10 ** (0.366 - 3.067 * R_4 + 1.930 * (R_4**2) + 0.649 * (R_4**3) - 1.532 * (R_4**4))',
        {
            'R_4': image.select(['B1', 'B2']).reduce(ee.Reducer.max()).divide(image.select('B3')).log10()
        }
    ))
    
    return oc4_collection

In [100]:
'''
NO recomendamos correr esta celda. Su ejecución puede tardar más de 1 hora.
En el archivo 'indices.csv' se encuentra el resultado de éstos cómputos.
'''
import os


result_dict = {fecha: '' for fecha in fechas}
count = 0
for fecha in fechas:
    print(fecha)    
    print(f'{len(fechas) - count} days remaining.')
    count+=1
    dayOfInterest = ee.Date(fecha)

    if os.path.isfile(f'MODIS/{fecha}.shp'):
        modis = gpd.read_file(f'MODIS/{fecha}.shp')
        
        sentinel2_collection = ee.ImageCollection('COPERNICUS/S2_SR')\
        .filterDate(dayOfInterest, dayOfInterest.advance(1, 'day'))\
        .filterBounds(aoi)

        evi = calculate_evi(sentinel2_collection).first()
        nrvi = calculate_nrvi(sentinel2_collection).first()
        ndwi = calculate_ndwi(sentinel2_collection).first()
        gndvi = calculate_gndvi(sentinel2_collection).first()
        mndwi = calculate_mndwi(sentinel2_collection).first() # 20m
        ndvi2 = calculate_ndvi2(sentinel2_collection).first() # 20m
        ndvi = calculate_ndvi(sentinel2_collection).first()
        b3b1 = calculate_b3b1(sentinel2_collection).first() # 60m
        b3b2 = calculate_b3b2(sentinel2_collection).first()
        b3b4 = calculate_b3b4(sentinel2_collection).first()
        b5b4 = calculate_b5b4(sentinel2_collection).first() # 20m.
        b6b4 = calculate_b6b4(sentinel2_collection).first() # 20m. 
        b7b4 = calculate_b7b4(sentinel2_collection).first() # 20m.
        b8ab4 = calculate_b8ab4(sentinel2_collection).first() # 20m.
        b8b4 = calculate_b8b4(sentinel2_collection).first()
        oc2 = calculate_oc2(sentinel2_collection).first()
        oc4 = calculate_oc4(sentinel2_collection).first()

        m_20 = {'mndwi': mndwi, 'ndvi2':ndvi2, 'b5b4': b5b4, 'b6b4': b6b4,
                'b7b4': b7b4, 'b8ab4': b8ab4, 'b8b4': b8b4}
        
        m_10 = {'evi': evi, 'nrvi': nrvi, 'ndwi': ndwi, 'gndvi': gndvi,
                'ndvi':ndvi, 'b3b2': b3b2, 'b3b4': b3b4, 'b8b4': b8b4,
            'oc2': oc2}

        m_60 = {'b3b1': b3b1, 'oc4': oc4}
        

        for metric in m_20.keys():
            print(f'Calculating {metric}')        
            geemap.ee_export_image(m_20[metric], filename='temp.tif', scale=20,region=aoi)
            temp = rioxarray.open_rasterio('temp.tif')
            temp = temp.rio.reproject("EPSG: 4326")
            # clip = temp.rio.clip([lagos.iloc[0,:].geometry])
            clip = temp
            metrics = []
            for i in modis.index:
                local_clip = clip.rio.clip([modis.iloc[i,:].geometry])
                metrics.append(local_clip.values.mean())
            to_append = pd.Series(metrics)
            modis[metric] = metrics
            temp.close()
            
        for metric in m_10.keys():
            print(f'Calculating {metric}')
            geemap.ee_export_image(m_10[metric], filename='temp.tif', scale=10,region=aoi)
            temp = rioxarray.open_rasterio('temp.tif')
            temp = temp.rio.reproject("EPSG: 4326")
            # clip = temp.rio.clip([lagos.iloc[0,:].geometry])
            clip = temp
            metrics = []
            for i in modis.index:
                local_clip = clip.rio.clip([modis.iloc[i,:].geometry])
                metrics.append(local_clip.values.mean())
            to_append = pd.Series(metrics)
            modis[metric] = metrics
            temp.close()
            
        for metric in m_60.keys():
            print(f'Calculating {metric}')        
            geemap.ee_export_image(m_60[metric], filename='temp.tif', scale=60,region=aoi)
            temp = rioxarray.open_rasterio('temp.tif')
            temp = temp.rio.reproject("EPSG: 4326")
            # clip = temp.rio.clip([lagos.iloc[0,:].geometry])
            clip = temp
            metrics = []
            for i in modis.index:
                local_clip = clip.rio.clip([modis.iloc[i,:].geometry])
                metrics.append(local_clip.values.mean())
            to_append = pd.Series(metrics)
            modis[metric] = metrics
            temp.close()
            
        modis['Fecha'] = fecha
        result_dict[fecha] = modis   
        

2019-08-19
13 days remaining.
2019-02-03
12 days remaining.
2020-04-08
11 days remaining.
2020-11-19
10 days remaining.
2022-02-22
9 days remaining.
2021-12-09
8 days remaining.
2020-03-01
7 days remaining.
2021-02-07
6 days remaining.
2022-01-28
5 days remaining.
2021-03-14
4 days remaining.
2020-02-20
3 days remaining.
2021-10-30
2 days remaining.
2019-03-10
1 days remaining.
